In [1]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 122.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 110.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 43.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.7 MB/s eta 0:00:00


In [2]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

import pyarrow as pa
import pyarrow.dataset as ds
from datasets import Dataset
import pandas as pd

In [3]:



model_name = "meta-llama/Llama-2-7b-chat-hf"

new_model = "Llama-2-7b-chat-finetune"



lora_r = 64
lora_alpha = 16
lora_dropout = 0.1


use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False



output_dir = "./results"

num_train_epochs = 1

# Look at you GPU, Ampere does support BF16, FP16 is on some other GPUs faster 
fp16 = False
bf16 = False


per_device_train_batch_size = 4
per_device_eval_batch_size = 4
gradient_accumulation_steps = 1
gradient_checkpointing = True
max_grad_norm = 0.3

learning_rate = 2e-4
weight_decay = 0.001
optim = "paged_adamw_32bit"
lr_scheduler_type = "cosine"

max_steps = 64
warmup_ratio = 0.03

group_by_length = True

save_steps = 0
logging_steps = 5

max_seq_length = None
packing = False

device_map = {"": 0} # Assign to you GPU

In [4]:
def format_dataset(df, use_context=False):
    df.fillna("")

    instruction_list = df["instruction"]
    #context_list = df["context"]
    context = "The following is something about the DHBW"
    response_list = df["response"]

    text_list = []

    if use_context:
        for instruction, response in zip(instruction_list, response_list):
            text = "<s>[INST] <<SYS>> " + context + " <</SYS>> " + instruction + " [/INST] " + response + " </s>"
            text_list.append(text)
    else:
        for instruction, response in zip(instruction_list, response_list):
            text = "<s>[INST] " + instruction + " [/INST] " + response + " </s>"
            text_list.append(text)

    df_text = pd.DataFrame(columns=["text"])
    df_text["text"] = text_list



    dataset = ds.dataset(pa.Table.from_pandas(df_text).to_batches())

    formated_dataset = Dataset(pa.Table.from_pandas(df_text))

    return formated_dataset

In [5]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [6]:
dataset = format_dataset(pd.read_csv("./dataset.csv"), use_context=True)

compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8: # 8 == Ampere
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" 

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

trainer.train()

trainer.model.save_pretrained(new_model)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/7 [00:00<?, ? examples/s]

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
5,3.557300
10,3.267300
15,2.474700
20,2.067100
25,1.634600
30,1.300900
35,0.996300
40,0.849900
45,0.633400
50,0.529800


In [14]:
prompt = "How do I access my DHBW E-Mail?"
context = "You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information. You are an IT Chatbot for an university (DHBW) and should just answer questions regarding DHBW or IT. Please answer always in German"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=600)
result = pipe(f"<s>[INST] <<SYS>> {context} <</SYS>> {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] <<SYS>> You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information. You are an IT Chatbot for an university (DHBW) and should just answer questions regarding DHBW or IT. Please answer always in German <</SYS>> How do I access my DHBW E-Mail? [/INST]  To access your DHBW e-mail, you can use the following options:

1. Web interface:

You can access your DHBW e-mail via the web at any time by using the following link: <https://webmail.dhbw.edu/>

2. DHBW ITM:

In addition to the web interface, you can also access your e-mail via the DHBW ITM. To do this, select the e-mail client of your choice (e.g. Thunderbird, Outloo

In [11]:
result[0]['generated_text']

'<s>[INST] <<SYS>> You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don\'t know the answer to a question, please don\'t share false information. You are an IT Chatbot for an university (DHBW) and should just answer questions regarding DHBW or IT. Please answer always in German <</SYS>> How can I acess my grades? [/INST]  As an IT chatbot, I\'ll do my best to assist you. To access your grades at DHBW Ravensburg, you can follow these steps:\n\n1. Open a web browser on your computer or mobile device.\n2. Enter the following URL and press Enter: <https://my.dhbw.edu/students/login>\n3. Enter your username and password.\n4. Click on "Grades" in the menu on the left.\n\nIf you have any question

In [12]:
import gc

del model, tokenizer
gc.collect()
torch.cuda.empty_cache()

In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:

model.push_to_hub("jeremistderechte/Llama-2-7b-chat-finetune", create_pr=1)

tokenizer.push_to_hub("jeremistderechte/Llama-2-7b-chat-finetune",create_pr=1)


CommitInfo(commit_url='https://huggingface.co/jeremistderechte/Llama-2-7b-chat-finetune/commit/37416a52ff5e9e52fbdab36f7045d236d732bec8', commit_message='Upload tokenizer', commit_description='', oid='37416a52ff5e9e52fbdab36f7045d236d732bec8', pr_url='https://huggingface.co/jeremistderechte/Llama-2-7b-chat-finetune/discussions/2', pr_revision='refs/pr/2', pr_num=2)